In [1]:
import numpy as np
import tensorflow as tf

import argparse
import time
import os

from model import *
from importlib import reload
from dataloader import DataProcess
from sample import *
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

In [2]:
def init_args():
    
        args = {}
        args['rnn_size'] = 100 
        args['tsteps'] = 300 
        args['batch_size'] = 32 
        args['num_batches'] = 500 
        args['num_mixtures'] = 20 # number of MDN mixtures
        args['learning_rate'] = 0.001
        args['epochs'] = 2500 
        args['alphabet'] = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
        args['tsteps_per_ascii'] = 25
        
        args['biases'] = 1.0
        args['logs_dir'] = './logs/'
        args['save_path'] = 'model9/model.ckpt' # path to save the model at
        args['load_path'] = 'model8/model.ckpt' # path to load the model from
        args['grad_clip'] = 10
        args['n_to_save'] = 500 #step difference at which to save the model
        args['scale_factor'] = 20
        args['gap'] = 500 #remove data with gap greater than this threshhold
        args['learning_rate_decay'] = 0.99 
        args['keep_prob'] = 0.85 # keep_prob for dropout
        args['train'] = False
        args['decay'] = 0.95
        args['momentum'] = 0.9
        return args

def load_pretrained_model(model, path):
        global_step = 0
        try:
            save_dir = '/'.join(path.split('/')[:-1])
            ckpt = tf.train.get_checkpoint_state(save_dir)
            load_path = ckpt.model_checkpoint_path
            model.saver.restore(model.sess, load_path)
            #load_was_success = True
        except Exception as e:
            print(e)
            load_was_success = False
        else:
            model.saver = tf.train.Saver(tf.global_variables())
            global_step = int(load_path.split('-')[-1])
            load_was_success = True
        return load_was_success, global_step
    
def train_model():
    args = init_args()
    args['train'] = True
    data_loader = DataProcess(args)
    
    # num_batches is calculated in dataloader based on total data size and batch_size
    args['num_batches'] = data_loader.num_batches
    print("num_bacthes", args['num_batches'])
    
    model = Model(args)
    load_was_success, global_step = load_pretrained_model(model, args['save_path'])
    
    # list to hold the loss values in each itration of the mini batch
    plot_loss = []
    model.sess.run(tf.assign(model.decay, args['decay']))
    model.sess.run(tf.assign(model.momentum, args['momentum']))
    
    # Mini batch for given number of epochs
    for e in range(int(global_step/args['num_batches']), args['epochs']):
        print("Running epoch", e)
        
        # learning rate decay
        model.sess.run(tf.assign(model.learning_rate, args['learning_rate'] * (args['learning_rate_decay'] ** e)))
        
       
        # initializes data pointer to starting of batch in each epoch
        data_loader.init_batch_comp()
        c0, c1, c2 = model.istate_cell0.c.eval(), model.istate_cell1.c.eval(), model.istate_cell2.c.eval()
        h0, h1, h2 = model.istate_cell0.h.eval(), model.istate_cell1.h.eval(), model.istate_cell2.h.eval()

        for b in range(global_step%args['num_batches'], args['num_batches']):

            i = e * args['num_batches'] + b
            if global_step is not 0 : i+=1 ; global_step = 0

            if i % args['n_to_save'] == 0 and (i > 0):
                # save the model we have right now
                model.saver.save(model.sess, args['save_path'], global_step = i) ;
            
            # get next batch of data to train on
            x, y = data_loader.get_next_batch()
            
            feed = {model.input: x, model.output: y, \
                    model.istate_cell0.c: c0, model.istate_cell1.c: c1, model.istate_cell2.c: c2, \
                    model.istate_cell0.h: h0, model.istate_cell1.h: h1, model.istate_cell2.h: h2}
            [train_loss, _] = model.sess.run([model.cost, model.train_op], feed)

            plot_loss.append(train_loss)
        print("train_loss: " + str(i))
        print(train_loss)
    
    # plotting the loss graph
    plt.plot(plot_loss, linewidth=2.0)
    plt.savefig("./loss.png")

# Function to sample some handwriting, this does not try to sample any particular style YET
def sample_model():
    args = init_args()
    args['tsteps'] = 1
    args['batch_size'] = 1

    model = Model(args)
    
    # load a pretrained model
    load_was_success, global_step = load_pretrained_model(model, args['load_path'])
    if load_was_success:
            strokes, char_to_plot = sample(model, args)
            line_plot_coef(strokes, 'Line plot', figsize = (20,4), save_path="./coef_plot7.png")
            line_plot_char(strokes, char_to_plot, 'Line plot', figsize = (20,4), save_path="./line_char7.png")
            print("plotted")

    else:
        print("Model failed to load, can't sample")


            
            
            
            
            
            
            
            
            
    

In [3]:
sample_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model8/model.ckpt-4000


/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/Users/saumyap/CS230_writing_synthesis/sample.py:36: RuntimeWarning: invalid value encountered in true_divide
  pi[0] = np.exp(pi_hat[0]) / np.sum(np.exp(pi_hat[0]), axis=0) # softmax
/Users/saumyap/CS230_writing_synthesis/sample.py:39: RuntimeWarning: invalid value encountered in less
  idx = np.random.choice(pi.shape[1], p=pi[0])
/Users/saumyap/CS230_writing_synthesis/sample.py:13: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  x = np.random.multivariate_normal(mean, cov, 1)


[13.766626, 1.7147443, 0.11929148, 0.30898774, -0.29390553, 3.429034e-08]
[-0.08373608, 0.08792426, 0.023241485, 0.03518726, 0.8840148, 1.1134836e-13]
[-0.0073569417, 0.07928049, 0.012878283, 0.00698189, 0.35872906, 7.960638e-07]
[0.038444243, 0.0446759, 0.008439568, 0.0053567607, 0.2961339, 0.003669816]
[0.0013144612, 0.048895612, 0.007675935, 0.0058531533, 0.14476027, 0.004447113]
[0.049193315, -0.011476773, 0.0060056844, 0.004223635, -0.06838681, 1.1432198e-06]
[-0.14164688, -0.015554711, 0.024000587, 0.044464346, 0.8334405, 1.09748495e-08]
[-0.14105244, -0.10119018, 0.023770623, 0.041766204, 0.8628093, 3.4519225e-08]
[-0.14261484, -0.19317357, 0.023461888, 0.044938073, 0.8639837, 7.777218e-08]
[-0.124519505, -0.31791055, 0.026383677, 0.04617716, 0.8601766, 2.2295446e-06]
[-0.05944109, -0.23840167, 0.04131167, 0.046734497, 0.87755066, 5.691992e-06]
[-0.04130979, -0.13408925, 0.043452457, 0.034950383, 0.84954304, 2.5400803e-07]
[-0.036178503, -0.13178094, 0.038916733, 0.024892977, 0.